In [2]:
import pandas as pd
import yfinance as yf

In [5]:
mtum = yf.Ticker("MTUM")

In [12]:
mtum.history()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-12-05,122.90,122.90,122.30,122.90,466700,0.000,0
2019-12-06,123.55,123.70,123.11,123.20,662000,0.000,0
2019-12-09,123.21,123.31,122.77,122.81,339800,0.000,0
2019-12-10,122.88,123.00,122.40,122.61,318000,0.000,0
2019-12-11,122.93,123.00,122.44,122.90,307900,0.000,0
2019-12-12,122.97,123.64,122.52,123.01,812200,0.000,0
2019-12-13,122.76,123.60,122.61,123.55,762300,0.000,0
2019-12-16,124.16,124.38,123.88,124.21,461200,0.505,0
2019-12-17,124.43,124.60,123.98,123.98,698100,0.000,0


In [15]:
from pandas_datareader import data as pdr
data = pdr.get_data_yahoo("MTUM", start="2017-01-01", end="2020-01-01")
data.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2017-01-03,76.459999,75.730003,76.129997,76.080002,284100.0,73.194923
2017-01-04,76.660004,76.239998,76.239998,76.529999,215400.0,73.627861
2017-01-05,76.820000,76.389999,76.540001,76.779999,393500.0,73.868370
2017-01-06,77.370003,76.599998,76.769997,77.220001,248900.0,74.291679
2017-01-09,77.360001,77.080002,77.250000,77.190002,158100.0,74.262848


In [16]:
len(data)

755

In [3]:
from pandas_datareader import data as pdr
import yfinance as yf
import datetime

yf.pdr_override() 

stocks = ["MTUM"]
start = datetime.datetime(2017,1,1)
end = datetime.datetime(2020,1,1)

f = pdr.get_data_yahoo(stocks, start=start, end=end)

[*********************100%***********************]  1 of 1 completed


In [4]:
len(f)

754

In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from pandas_datareader import data as pdr
import datetime
import matplotlib.pyplot as plt

In [2]:
#Cargamos ambos files y los unimos, usando como indices en ambos las fechas.
def load_and_join():
    mtum = yf.Ticker("MTUM")
    stocks = ["MTUM"]
    start = datetime.datetime(2000,11,30)
    end = datetime.datetime(2019,11,30)
    
    yf.pdr_override()
    
    df_etf = pdr.get_data_yahoo(stocks, start=start, end=end)
    
    df = pd.read_excel('https://images.aqr.com/-/media/AQR/Documents/Insights/Data-Sets/Century-of-Factor-Premia-Monthly.xlsx',
                      header =18, nrows = 1220)
    
    df['Date'] =  pd.to_datetime(df['Date'])
    
    df = df.set_index('Date')
    df = df[['Equity indices Momentum','Equity indices Momentum','Equity indices Carry','Equity indices Defensive']]
    
    df_final = df_etf.merge(df, how='left',left_index=True,right_index=True)
    
    return df_final

#Extendemos los datos mensuales a los registros diarios. (Ejemplo: todos los datos de enero, tomaran el valor monthly del 31/01)
def fill_joined_missing_fields(df):
    for i in range(1,len(df)+1):
        if np.isnan(df.iloc[-i,9]):
                df.iloc[-i,6] = df.iloc[-i+1,6]
                df.iloc[-i,7] = df.iloc[-i+1,7]
                df.iloc[-i,8] = df.iloc[-i+1,8]
                df.iloc[-i,9] = df.iloc[-i+1,9]
         

#Introducimos labels al dataset, calculando medias 50 y 200 dias.
'''(REVEER!)'''    
def labeling_df(df):
    df['50_days_average'] = df.iloc[:,3].rolling(window=50).mean()
    df['200_days_average'] = df.iloc[:,3].rolling(window=200).mean()
    df.loc[df['50_days_average'] >= df['200_days_average'], 'Buy/Sell'] = -1 #Si la media de corto plazo supera a la de largo, es posicion de sell
    df.loc[df['50_days_average'] < df['200_days_average'], 'Buy/Sell'] = 1 #Si la media de corto plazo esta por debajo de la de largo, es posicion de buy
    return df
 
#Visualizacion de la evolucion de precios y las medias.
def visualize_close_50_200(df):
    plt.plot(df['Close'])
    plt.plot(df['50_days_average'])
    plt.plot(df['200_days_average'])
    plt.legend(['Close','50_days_avg','200_days_avg'])
    plt.title('Evolution of MTUM ETF over time')
    plt.show()

#Consigue los weights para la diferenciacion!
def getWeights_FFD(d,size):
    w=[1.]
    for k in range(1,size):
        w_=-w[-1]/k*(d-k+1)
        w.append(w_)
    w=np.array(w[::-1]).reshape(-1,1)
    return w

#Funcion aux para pesos de FFD
def plotWeights(dRange,nPlots,size):
    w=pd.DataFrame()
    for d in np.linspace(dRange[0],dRange[1],nPlots):
        w_=getWeights_FFD(d,size=size)
        w_=pd.DataFrame(w_,index=range(w_.shape[0])[::-1],columns=[d])
        w=w.join(w_,how='outer')
    ax=w.plot()
    ax.legend(loc='upper right');plt.show()
    return

#Diferenciamos la serie! d es el orden de diferenciacion. "Thres" (threshold) maneja la acceptabilidad de las exclusiones. No modificar.
def fracDiff(series,d,thres=0.01):

    w=getWeights_FFD(d,series.shape[0])

    w_=np.cumsum(abs(w))
    w_/=w_[-1]
    skip=w_[w_>thres].shape[0]

    df={}
    for name in series.columns:
        seriesF,df_=series[[name]].fillna(method='ffill').dropna(),pd.Series()
        for iloc in range(skip,seriesF.shape[0]):
            loc=seriesF.index[iloc]

            df_[loc]=np.dot(w[-(iloc+1):,:].T,seriesF.loc[:loc])[0,0]
        df[name]=df_.copy(deep=True)
    df=pd.concat(df,axis=1)
    return df

#Funcion para buscar el mejor d
def plotMinFFD(df):
    from statsmodels.tsa.stattools import adfuller
    import numpy.ma as ma
    out=pd.DataFrame(columns=['adfStat','pVal','lags','nObs','95% conf','corr'])
    for d in np.linspace(0,1,21):
        df1=np.log(df[['Close']]).resample('1D').last() # Pasar a observaciones diarias
        df2=fracDiff(df1,d,thres=.01)
        corr = ma.corrcoef(ma.masked_invalid(df1.loc[df2.index,'Close']), ma.masked_invalid(df2['Close']))[0,1]
        df2=adfuller(df2['Close'],maxlag=1,regression='c',autolag=None)
        out.loc[d]=list(df2[:4])+[df2[4]['5%']]+[corr] # Aportar valores criticos
    out[['adfStat','corr']].plot(secondary_y='adfStat')
    plt.axhline(out['95% conf'].mean(),linewidth=1,color='r',linestyle='dotted')
    plt.show()
    return out

#Obtiene los factores de tiempo para restar importancia a las observaciones.
def getTimeDecay(tW,clfLastW):
    clfW=tW.sort_index().cumsum()
    if clfLastW>=0:slope=(1.-clfLastW)/clfW.iloc[-1]
    else:slope=1./((clfLastW+1)*clfW.iloc[-1])
    const=1.-slope*clfW.iloc[-1]
    clfW=const+slope*clfW
    clfW[clfW<0]=0
    # print(const,slope)
    return clfW


In [12]:
df = load_and_join()

[*********************100%***********************]  1 of 1 completed


In [14]:
len(df)

1668

In [15]:
df.head(50)

,Open,High,Low,Close,Adj Close,Volume,Equity indices Momentum,Equity indices Momentum,Equity indices Carry,Equity indices Defensive
Date,,,,,,,,,,
2013-04-18,50.919998,50.919998,50.919998,50.919998,46.707516,400,NaN,NaN,NaN,NaN
2013-04-19,50.919998,50.919998,50.919998,50.919998,46.707516,0,NaN,NaN,NaN,NaN
2013-04-22,50.919998,50.919998,50.919998,50.919998,46.707516,0,NaN,NaN,NaN,NaN
2013-04-23,52.220001,52.220001,52.220001,52.220001,47.899956,600,NaN,NaN,NaN,NaN
2013-04-24,51.700001,51.700001,51.560001,51.560001,47.294567,1100,NaN,NaN,NaN,NaN
2013-04-25,51.810001,51.810001,51.810001,51.810001,47.523891,400,NaN,NaN,NaN,NaN
2013-04-26,51.770000,51.770000,51.770000,51.770000,47.487190,100,NaN,NaN,NaN,NaN
2013-04-29,51.849998,52.200001,51.849998,52.070000,47.762371,4800,NaN,NaN,NaN,NaN
2013-04-30,51.619999,51.930000,51.619999,51.840000,47.551403,1400,-0.011301,-0.011301,0.006270,-0.027077


In [6]:
fill_joined_missing_fields(df)  

In [8]:
df.head(30)

,Open,High,Low,Close,Adj Close,Volume,Equity indices Momentum,Equity indices Momentum,Equity indices Carry,Equity indices Defensive
Date,,,,,,,,,,
2013-04-18,50.919998,50.919998,50.919998,50.919998,46.707516,400,-0.011301,-0.011301,0.006270,-0.027077
2013-04-19,50.919998,50.919998,50.919998,50.919998,46.707516,0,-0.011301,-0.011301,0.006270,-0.027077
2013-04-22,50.919998,50.919998,50.919998,50.919998,46.707516,0,-0.011301,-0.011301,0.006270,-0.027077
2013-04-23,52.220001,52.220001,52.220001,52.220001,47.899956,600,-0.011301,-0.011301,0.006270,-0.027077
2013-04-24,51.700001,51.700001,51.560001,51.560001,47.294567,1100,-0.011301,-0.011301,0.006270,-0.027077
2013-04-25,51.810001,51.810001,51.810001,51.810001,47.523891,400,-0.011301,-0.011301,0.006270,-0.027077
2013-04-26,51.770000,51.770000,51.770000,51.770000,47.487190,100,-0.011301,-0.011301,0.006270,-0.027077
2013-04-29,51.849998,52.200001,51.849998,52.070000,47.762371,4800,-0.011301,-0.011301,0.006270,-0.027077
2013-04-30,51.619999,51.930000,51.619999,51.840000,47.551403,1400,-0.011301,-0.011301,0.006270,-0.027077
